In [4]:
pip install pydirectinput

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mediapipe as mp
import cv2
import pydirectinput

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_pose=mp.solutions.pose

In [4]:
cap = cv2.VideoCapture(0)
cap.set(3,560)
cap.set(4,400)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,image=cap.read()
        if not ret:
            break
        img=cv2.flip(image,1)
        frame=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        results=holistic.process(frame)
        frame=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        height,width,_=image.shape
        y_mid=height//2
        pose="move"
        
        try:
            if results.pose_landmarks:
                left_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
                right_hand_y = left_wrist.y * height

                if right_hand_y < y_mid:
                    pose = "acc"
                    pydirectinput.keyDown('right')
                    pydirectinput.keyUp('left')
                elif right_hand_y > y_mid:
                    pose = "brake"
                    pydirectinput.keyUp('right')
                    pydirectinput.keyDown('left')
        except Exception as e:
            print("Error:", e)

        cv2.putText(img, pose, (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)
        cv2.line(img, (0, y_mid), (width, y_mid), (255, 0, 255), 2)
        cv2.imshow('Hill climb racing', img)

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()